In [ ]:
-- Create a view that combines traffic accidents, vehicle stock, and weather data
-- Baseline: traffic_accidents table (all records start here)
CREATE VIEW traffic_analysis_view AS
SELECT 
    -- ====================
    -- Time Dimensions
    -- ====================
    a.year_month,
    a.year,
    a.month,
    
    -- ====================
    -- Location
    -- ====================
    a.prefecture,
    
    -- ====================
    -- Traffic Accidents
    -- ====================
    a.accidents,
    a.deaths,
    a.injuries,
    
    -- ====================
    -- Car Stock Data
    -- ====================
    cs.kei_passenger,
    cs.kei_cargo,
    cs.kei_total,
    cs.registered_vehicles,
    cs.all_vehicles,
    
    -- ====================
    -- Weather Data
    -- ====================
    w.precip_total_mm,
    w.precip_max_daily_mm,
    w.temp_avg_c,
    w.temp_max_c,
    w.temp_min_c,
    w.wind_speed_avg_ms,
    w.wind_speed_max_ms,
    w.snowfall_total_cm,
    w.snow_depth_max_cm,
    w.snow_days,
    w.fog_days,
    
    -- ====================
    -- Calculated Rates (per vehicle)
    -- ====================
    CASE 
        WHEN cs.all_vehicles > 0 
        THEN (a.accidents * 1.0) / cs.all_vehicles 
        ELSE NULL 
    END AS acc_rate,
    
    CASE 
        WHEN cs.all_vehicles > 0 
        THEN (a.deaths * 1.0) / cs.all_vehicles 
        ELSE NULL 
    END AS mortality_rate,
    
    CASE 
        WHEN cs.all_vehicles > 0 
        THEN (a.injuries * 1.0) / cs.all_vehicles 
        ELSE NULL 
    END AS morbidity_rate

-- Start with traffic accidents as the baseline
FROM traffic_accidents a

-- Add car stock data (LEFT JOIN = keep all accidents even if no matching car data)
LEFT JOIN car_stock_monthly_prefecture_estimated cs
    ON (a.prefecture = cs.prefecture 
        AND a.year_month = cs.year_month)

-- Add weather data (LEFT JOIN = keep all accidents even if no matching weather data)
LEFT JOIN weather_data_prefecture w
    ON (a.prefecture = w.prefecture 
        AND a.year_month = w.year_month)